## Text Miner

In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction
from nltk.stem.snowball import SnowballStemmer
import re
stemmer = SnowballStemmer("italian")
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:,.0f}'.format

In [3]:
#carico il file
data = pd.read_excel("C:\\Users\\Family\\Desktop\\Ema\\Python\\Dataset.xlsx")
print(data.shape)

(45713, 31)


In [4]:
data = data[["NPS(TM)", "Verbatim"]]
data.head()

,NPS(TM),Verbatim
0,8,facilità d’uso
1,8,Procedere con app o home banking è abbastanza ...
2,9,Tutto
3,8,Perché sino più di 40 anni che la mia famiglia...
4,4,Dipendenti non sempre gentili. Competitor più...


In [5]:
def NPScluster(x):
    if (x<7):
        x = 'Detrattore'
    elif ((x>6) & (x<9)):
        x = 'Neutrale'
    else:
        x = 'Promotore'
    return x

def tokenize_and_stem(text):
    text = text.lower()
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

#metodo di matching
def matchCompost(verbatim,parole):
    import re
    result = False
    i = 0
    while(i<len(parole)):
        if(re.search(parole[i],str(verbatim), re.IGNORECASE)):
            result = True
            i+=1
        else: 
            i+=1
    return result

#1
def creaColonneDriverVerbatim(data,driver):
    lenDriver = len(driver)
    for i in range(lenDriver):
        data = riempiColonneDriverVerbatim(data,driver[i])
    return data

#2    
def riempiColonneDriverVerbatim(data, driver):
    data[driver] = data['Verbatim'].apply(lambda x: driver01(str(x),driver))
    return data

#3
def driver01(verbatim, driver):
    dizionario = corrispondenzeDriver(verbatim,driver)
    if dizionario.get(driver) == driver:
        return 1
    else: 
        return 0
#4
def corrispondenzeDriver(verbatim,driver):
    d0="" #rabbia
    d1="" #disgusto
    d2="" #tristezza
    d3="" #gioia
    d4="" #paura
    d5="" #sorpresa

    cont=0
    verbatim = verbatim.lower()
    
    l0=[" dissens"," non approv"," conflitt"," non d'accord", " inaccord"," divergenz"," lancinant"," mal di testa",
        " nessuna risposta", " non risponde nessuno"," nessuno rispond"," frustrat"," non riesco"," frustrante",
        " non si può"," non mi soddisf"," insoddisfatt"," inappagat"," afflitt"," macello"," casino"," lament"," denunc",
        " lamentel"," non funzion"," blocco"," arrabbiat"," mi arrabbiar"," incazza"," rott", " stanc"," ira"," non faccio più",
        " al limite"," non gentilezz" ," complessit"," peggi"," peggior"," poco professional" ,"assoluta mancanz"," complicat",
        " disappunto"," non possibil"," non soddi"," mai trov ben", "non ti aiut", " irrispett"," insoddisfat"]
    
    l1=[" mult"," sanzion"," scorrett"," inutil"," dev"," indignat"," pazzi"," assurd"," non tutelat"," schif"," merd",
        " disapprov"," ma che cazz", " che cavol"," disprezz"," vergogn"," pessim"," eccessiv"," ridicol"," scandalos",
        " scadent"]
    
    l2=[" amareggiat"," delus"," fastid"," sfiduci"," preoccupat"," senza fiducia"," non ho fiducia"," contrariat",
        " quando riprenderà", " spento"," trist"," malinconic"," pensieros"," passat"," megli prim"," inquiet"," scocciant",
        " non so cos far"," annoiat"," noia"," deprim"," depriment"," stess cos"," ripetitiv"," routin"," nostalgi"," a pezz",
        " giù corda"," infelic"," disperat"," tristezz"," piangere", " abbattut"," amarezz"," abbandon"," scontent", 
        " dispiac"," mancanz"," manca"]
    
    l3=[" vicin"," sentiment"," affezionat"," benevolenz"," tenerezz"," affett"," attaccament"," affettuosit"," calor",
        " mi siete vicini"," vicin"," sostegn"," prezios"," protegge"," amiciz"," amic"," friend"," ammirazion"," ammir"," amar",
        " tutel"," creder"," fidat"," fidar"," mi fido"," fiduc"," riconoscenz"," grato", "grata"," gratitudin"," facilità d'us",
        " trovo ben", " soddisfaz"," soddisfatt"," stimat"," considerat"," d'accord"," rispett"," tutel"," mai delus"," ottim",
        " mai lament"," non mi mai delus", " non mi delud"," aiut"]
    
    l4=[" drammatic"," fa tremar"," moment difficolt"," moment difficil"," non facil"," emergenz"," non sicur"," angosc",
        " depress", " groppo in gola"," ansi"," oppressio"," oppress"," panico"," disperat"," timor"," tem"," paur"," teme",
        " truff"," difficolt"," difficil", " incub"," non vicin"," nonostant", " preoccup", ]
    
    l5=[" entusiast"," eccitazion"," appagat"," foga"," sorpres"," oltre le aspettativ"," accoglienz"," stupid"," stupor",
        " prontezz"," risolve"," risolt", " soluzion"," scopert"," agi"]
    
    if(driver=="rabbia"):
        if matchCompost(verbatim, l0):
            d0="rabbia"
            cont = +1
    if(driver=="disgusto"):
        if matchCompost(verbatim, l1):
            d1="disgusto"
            cont = +1
    if(driver=="tristezza"):
        if matchCompost(verbatim, l2):
            d2="tristezza"
            cont = +1
    if(driver=="gioia"):
        if matchCompost(verbatim, l3):
            d3="gioia"
            cont = +1
    if(driver=="paura"):
        if matchCompost(verbatim, l4):
            d4="paura"
            cont = +1
    if(driver=="sorpresa"):
        if matchCompost(verbatim, l5):
            d5="sorpresa"
            cont = +1
    
    dictDriver = {"rabbia":d0,
                "disgusto":d1,
                "tristezza":d2,
                "gioia":d3,
                "paura":d4,
                "sorpresa":d5}
    # print(cont)
    return dictDriver


In [6]:
drivers = ["rabbia",
          "disgusto",
          "tristezza",
          "gioia",
          "paura",
          "sorpresa"]

In [7]:
%%time
#data = data[data.Verbatim.str.contains("app ")]
dataWork = creaColonneDriverVerbatim(data,drivers)
dataWork['generico'] = dataWork.loc[:,"rabbia":].sum(axis=1)
dataWork['generico'] = dataWork.generico.apply(lambda x: 1 if x==0 else 0)
dataWork.head()

Wall time: 24.7 s


,NPS(TM),Verbatim,rabbia,disgusto,tristezza,gioia,paura,sorpresa,generico
0,8,facilità d’uso,0,0,0,0,0,0,1
1,8,Procedere con app o home banking è abbastanza ...,0,0,0,0,0,0,1
2,9,Tutto,0,0,0,0,0,0,1
3,8,Perché sino più di 40 anni che la mia famiglia...,0,0,0,1,0,0,0
4,4,Dipendenti non sempre gentili. Competitor più...,0,0,0,0,0,0,1
